<a href="https://colab.research.google.com/github/millacurafa/AdvancedDataScience/blob/main/Ejemplo_PyCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 11.1MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620777 sha256=3098af469269f226a84ad28a9d29a572550ff044161136ecaf2f698d578e4e93
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=fc569958be3b3241ad1cdaa2236239658e196819e329dc8bf7060a8fa00d52c8
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

#Se crea matriz de 4x4 con números del 0 al 9
a = numpy.random.randint(10, size=(8,8))
#Se convierte el tipo de dato de int64 a int32
a = a.astype(numpy.int32)

#Se asigna la la memoria en la GPU con la cantidad de bytes que ocupa la matriz a 
a_gpu = cuda.mem_alloc(a.nbytes)
#Se realiza la copia de los datos de host -> device
cuda.memcpy_htod(a_gpu, a)

#Kernel CUDA, es la función que va a ejecutar cada hebra instanciada
# Documento para índices de hebras https://cs.calvin.edu/courses/cs/374/CUDA/CUDA-Thread-Indexing-Cheatsheet.pdf

# Para una GTX 1080
# Maximum number of threads per block:           1024
# Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
# Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)


mod = SourceModule("""
   __global__ void doublify(int *a)
   {
  
     int blockId = blockIdx.x + blockIdx.y * gridDim.x;
     int idx = blockId * (blockDim.x * blockDim.y) + (threadIdx.y * blockDim.x) + threadIdx.x;

     a[idx] *= 2;
   }
   """)

#Convierte el kernel a una función en python
func = mod.get_function("doublify")
#Se ejecuta el kernel, con bloques de tamaño 4x4x1

start = cuda.Event()
end   = cuda.Event()

start.record()
func(a_gpu, block=(16,16,1),grid=(2,2,1))
end.record()
end.synchronize()

millis = start.time_till(end)
print("Tiempo: " + str(millis))
#Devuelve un nuevo arreglo con la misma forma y tipo del arreglo dado
a_doubled = numpy.empty_like(a)

#Se realiza la copia de los datos de device -> host
cuda.memcpy_dtoh(a_doubled, a_gpu)

print(a)
print()
print(a_doubled)
print()


Tiempo: 0.088128000497818
[[9 9 5 6 5 0 6 7]
 [1 1 4 4 0 3 6 3]
 [0 9 1 5 1 6 6 8]
 [6 4 1 4 7 3 7 5]
 [8 2 0 7 3 6 9 7]
 [8 9 5 5 3 8 9 6]
 [3 2 3 7 2 9 1 9]
 [4 2 6 8 7 0 7 8]]

[[18 18 10 12 10  0 12 14]
 [ 2  2  8  8  0  6 12  6]
 [ 0 18  2 10  2 12 12 16]
 [12  8  2  8 14  6 14 10]
 [16  4  0 14  6 12 18 14]
 [16 18 10 10  6 16 18 12]
 [ 6  4  6 14  4 18  2 18]
 [ 8  4 12 16 14  0 14 16]]

